In [ ]:
! cat /proc/cpuinfo

In [20]:
! nvidia-smi

Tue Mar 22 15:25:48 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P0    75W / 149W |     58MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

In [10]:
!nvcc --version


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [12]:
!pip install pycuda # install cuda
import pycuda.driver as cuda
import pycuda.autoinit

from pycuda.compiler import SourceModule

     |████████████████████████████████| 1.7 MB 5.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 67 kB 5.8 MB/s 
     |████████████████████████████████| 78 kB 6.4 MB/s 
  Created wheel for pycuda: filename=pycuda-2021.1-cp37-cp37m-linux_x86_64.whl size=626634 sha256=d9eb07a38bfd0c62021068debfcd4926a2b95f96aa414953f9dfc8095461673d
  Stored in directory: /root/.cache/pip/wheels/c4/ef/49/dc6a5feb8d980b37c83d465ecab24949a6aa19458522a9e001
  Created wheel for pytools: filename=pytools-2022.1.2-py2.py3-none-any.whl size=63903 sha256=72450ddc0ef30cbcc7afb79ec3e9c368a40a90a439e4efbeb95dc69cd009dd11
  Stored in directory: /root/.cache/pip/wheels/71/24/3d/aa7df2a581a0cc0c86ca368104c8e3613848c78df40a88e9f9
Successfully built pycuda pytools


In [16]:
mod = SourceModule("""
__global__ void array_sum(float dest, float *arr)
{
  const int i = threadIdx.x;
  dest+=arr[i];
}
""")

In [17]:
import numpy as np
np.random.seed(1000)
size = 50
a = [0]*size

for i in range(size):
  a[i] = np.random.randn(0,50)


In [23]:
sum_array = mod.get_function("array_sum")

b = numpy.random.randn(400).astype(numpy.float32)
dest = 0
sum_array(
    cuda.Out(dest), cuda.In(b),
    block=(400,1,1), grid=(1,1))

print(dest)

TypeError: ignored

In [31]:
mod = SourceModule("""
__global__ void multiply_them(float dest, float *a)
{
  const int i = threadIdx;
  dest +=  a[i];
}
""")

multiply_them = mod.get_function("multiply_them")

a = numpy.random.randn(50).astype(numpy.float32)

dest = 0
multiply_them(
        cuda.Out(dest), cuda.In(a),
        block=(50,1,1), grid=(1,1))

print(dest-a*b)

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



CompileError: ignored